In [1]:
import os
import csv
import random
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
google_apps = pd.read_csv("googleplaystore.csv")
google_apps.shape

(10841, 13)

In [3]:
google_apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,07-Jan-18,1.00.00,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.00.00,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,01-Aug-18,1.02.04,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,08-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


In [4]:
google_apps['Rating'].fillna((google_apps['Rating'].median()), inplace=True)

In [5]:
google_apps['Type']= google_apps['Type'].fillna(google_apps['Type'].value_counts().index[0])
google_apps['Content Rating']= google_apps['Content Rating'].fillna(google_apps['Content Rating'].value_counts().index[0])
google_apps=google_apps.dropna()

In [6]:
#DATA CLEANING

#total number of unique apps in dataset
no_of_apps = google_apps["App"].nunique()
print(f"Number of unique apps: {no_of_apps}")

#remove all duplicate rows corresponding to app column
google_apps.drop_duplicates(subset=['App'], keep=False)
google_apps

#delete unnecesaary rows creating issues in modelling
#google_apps = google_apps.drop([10472])

#conversion of size column to a convinient format
google_apps["Size"] = google_apps["Size"].apply(lambda x: str(x).replace(",", "") if "," in str(x) else x)
google_apps["Size"] = google_apps["Size"].apply(lambda x: str(x).replace('M', '') if 'M' in str(x) else x)
google_apps["Size"] = google_apps["Size"].apply(lambda x: str(x).replace("Varies with device", "NAN") if "Varies with device" in str(x) else x)
google_apps["Size"] = google_apps["Size"].apply(lambda x: float(str(x).replace('k', '')) / 1024 if 'k' in str(x) else x)
google_apps["Size"] = google_apps["Size"].apply(lambda x:x.replace("+","")if "+" in str(x) else x)

#convert size to float and replace 'NAN' in size column with mean of the column
google_apps["Size"] = google_apps["Size"].apply(lambda x:float(x))
google_apps["Size"].fillna((google_apps["Size"].mean()), inplace=True)


#Install column with '+', ',' signs are removed to help easy computation
google_apps['Installs']=google_apps['Installs'].str.replace(',','').str.replace('+','').astype('float')


#Converting Type column to 1 (for paid) and 0 (for free) apps
def type_cat(types):
    if types == 'Free':
        return 0
    else:
        return 1
google_apps['Type'] = google_apps['Type'].map(type_cat)
    
google_apps["Reviews"] = google_apps["Reviews"].astype("int64")
    
#Make the price column consistent by removing the '$' symbol and replacing "Free" with 0
google_apps["Price"] = google_apps["Price"].apply(lambda x: x.replace("Free",0) if "Free" in str(x) else x)
google_apps["Price"] = google_apps["Price"].apply(lambda x:x.replace("$","") if "$" in str(x)else x)
google_apps["Price"] = google_apps["Price"].apply(lambda x: float(x))
google_apps["Price"].dtype

google_apps=google_apps.drop_duplicates(subset='App', keep='first')

Number of unique apps: 9649


C:\Users\Vamsi Sai\AppData\Local\Programs\Python\Python37-32\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [7]:
google_apps.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19.0,10000.0,0,0.0,Everyone,Art & Design,07-Jan-18,1.00.00,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14.0,500000.0,0,0.0,Everyone,Art & Design;Pretend Play,15-Jan-18,2.00.00,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7,5000000.0,0,0.0,Everyone,Art & Design,01-Aug-18,1.02.04,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25.0,50000000.0,0,0.0,Teen,Art & Design,08-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8,100000.0,0,0.0,Everyone,Art & Design;Creativity,20-Jun-18,1.1,4.4 and up


In [8]:
google_apps_copy = google_apps.copy()
label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in columns and converting the type
google_apps_copy['Category'] = label_encoder.fit_transform(google_apps_copy['Category']) 
google_apps_copy['Content Rating'] = label_encoder.fit_transform(google_apps_copy['Content Rating']) 
google_apps_copy['Genres'] = label_encoder.fit_transform(google_apps_copy['Genres']) 
google_apps_copy.dtypes

App                object
Category            int32
Rating            float64
Reviews             int64
Size              float64
Installs          float64
Type                int64
Price             float64
Content Rating      int32
Genres              int32
Last Updated       object
Current Ver        object
Android Ver        object
dtype: object

In [9]:
google_apps_copy

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,0,4.1,159,19.000000,10000.0,0,0.0,1,9,07-Jan-18,1.00.00,4.0.3 and up
1,Coloring book moana,0,3.9,967,14.000000,500000.0,0,0.0,1,12,15-Jan-18,2.00.00,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",0,4.7,87510,8.700000,5000000.0,0,0.0,1,9,01-Aug-18,1.02.04,4.0.3 and up
3,Sketch - Draw & Paint,0,4.5,215644,25.000000,50000000.0,0,0.0,4,9,08-Jun-18,Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,0,4.3,967,2.800000,100000.0,0,0.0,1,11,20-Jun-18,1.1,4.4 and up
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10836,Sya9a Maroc - FR,11,4.5,38,53.000000,5000.0,0,0.0,1,38,25-Jul-17,1.48,4.1 and up
10837,Fr. Mike Schmitz Audio Teachings,11,5.0,4,3.600000,100.0,0,0.0,1,38,06-Jul-18,1,4.1 and up
10838,Parkinson Exercices FR,20,4.3,3,9.500000,1000.0,0,0.0,1,70,20-Jan-17,1,2.2 and up
10839,The SCP Foundation DB fr nn5n,3,4.5,114,21.534596,1000.0,0,0.0,3,19,19-Jan-15,Varies with device,Varies with device


In [11]:
google_apps_copy = google_apps_copy.drop(["App","Last Updated","Current Ver","Android Ver"],axis=1)
google_apps_copy

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,0,4.1,159,19.000000,10000.0,0,0.0,1,9
1,0,3.9,967,14.000000,500000.0,0,0.0,1,12
2,0,4.7,87510,8.700000,5000000.0,0,0.0,1,9
3,0,4.5,215644,25.000000,50000000.0,0,0.0,4,9
4,0,4.3,967,2.800000,100000.0,0,0.0,1,11
...,...,...,...,...,...,...,...,...,...
10836,11,4.5,38,53.000000,5000.0,0,0.0,1,38
10837,11,5.0,4,3.600000,100.0,0,0.0,1,38
10838,20,4.3,3,9.500000,1000.0,0,0.0,1,70
10839,3,4.5,114,21.534596,1000.0,0,0.0,3,19


In [12]:
print("There are {} total rows.".format(google_apps_copy.shape[0]))
count_popular = google_apps_copy[google_apps_copy["Installs"] > 100000].count()

print("{} have High Install popularity !".format(count_popular[0]))
print("{} have Low Install popularity !\n".format((google_apps_copy.shape[0]-count_popular)[0]))

google_apps_copy["Installs"] = (google_apps_copy["Installs"] > 100000)*1 #Installs Binarized

There are 9649 total rows.
3898 have High Install popularity !
5751 have Low Install popularity !



In [13]:
testing_popular1 = google_apps_copy[google_apps_copy["Installs"] == 1].sample(1010)
google_apps_copy = google_apps_copy.drop(testing_popular1.index)
#print("Values were not dropped from training dataframe.",testing_popular1.index[0] in google_apps_copy.index)

testing_popular0 = google_apps_copy[google_apps_copy["Installs"] == 0].sample(766)
google_apps_copy = google_apps_copy.drop(testing_popular0.index)
#print("Values were not dropped from training dataframe.",testing_popular0.index[0] in google_apps_copy.index)

In [14]:
testing_popular1

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
1060,12,4.7,130582,24.000000,1,0,0.00,1,59
70,1,4.8,116986,35.000000,1,0,0.00,1,13
2574,27,4.0,22695,21.534596,1,0,0.00,2,100
336,6,4.4,69119316,21.534596,1,0,0.00,1,35
5490,28,4.4,129409,73.000000,1,1,1.99,1,101
...,...,...,...,...,...,...,...,...,...
2547,27,4.4,49173,4.000000,1,0,0.00,1,100
2352,20,4.3,2218,21.534596,1,0,0.00,1,70
1281,15,4.4,708674,21.534596,1,0,0.00,1,61
3225,30,4.2,6762,8.500000,1,0,0.00,1,109


In [15]:
train_google_apps = google_apps_copy

test_google_apps = testing_popular1.append(testing_popular0)

#Shuffle rows in test & training data set
test_google_apps = test_google_apps.sample(frac=1, random_state=0).reset_index(drop=True)

In [16]:
train_google_apps

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,0,4.1,159,19.000000,0,0,0.0,1,9
1,0,3.9,967,14.000000,1,0,0.0,1,12
3,0,4.5,215644,25.000000,1,0,0.0,4,9
4,0,4.3,967,2.800000,0,0,0.0,1,11
6,0,3.8,178,19.000000,0,0,0.0,1,9
...,...,...,...,...,...,...,...,...,...
10835,4,4.3,0,9.600000,0,0,0.0,1,22
10837,11,5.0,4,3.600000,0,0,0.0,1,38
10838,20,4.3,3,9.500000,0,0,0.0,1,70
10839,3,4.5,114,21.534596,0,0,0.0,3,19


In [17]:
test_google_apps

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres
0,30,4.3,165299,5.400000,1,0,0.0,1,109
1,6,3.8,12667,34.000000,1,0,0.0,1,35
2,19,3.0,4,21.534596,0,0,0.0,1,69
3,21,4.3,355921,17.000000,1,0,0.0,4,74
4,14,4.2,3816,31.000000,0,0,0.0,4,6
...,...,...,...,...,...,...,...,...,...
1771,6,4.5,305218,21.534596,1,0,0.0,1,35
1772,12,4.4,337,3.300000,0,0,0.0,1,59
1773,11,4.0,1542,13.000000,0,0,0.0,1,51
1774,30,3.9,13275,71.000000,1,0,0.0,1,109


In [18]:
y_train = train_google_apps.pop("Installs")
X_train = train_google_apps.copy()
y_test = test_google_apps.pop("Installs")
X_test = test_google_apps.copy()

In [19]:
y_train

0        0
1        1
3        1
4        0
6        0
        ..
10835    0
10837    0
10838    0
10839    0
10840    1
Name: Installs, Length: 7873, dtype: int32

In [21]:
X_train

,Category,Rating,Reviews,Size,Type,Price,Content Rating,Genres
0,0,4.1,159,19.000000,0,0.0,1,9
1,0,3.9,967,14.000000,0,0.0,1,12
3,0,4.5,215644,25.000000,0,0.0,4,9
4,0,4.3,967,2.800000,0,0.0,1,11
6,0,3.8,178,19.000000,0,0.0,1,9
...,...,...,...,...,...,...,...,...
10835,4,4.3,0,9.600000,0,0.0,1,22
10837,11,5.0,4,3.600000,0,0.0,1,38
10838,20,4.3,3,9.500000,0,0.0,1,70
10839,3,4.5,114,21.534596,0,0.0,3,19


In [29]:
print("{}% apps are used for Training.".format((y_train.count()/(y_train.count()+y_test.count()))*100))
print("{} apps are used for Testing.".format(y_test.count()/(y_train.count()+y_test.count())*100))
X_test.head()

81.59394755933256% apps are used for Training.
18.406052440667427 apps are used for Testing.


,Category,Rating,Reviews,Size,Type,Price,Content Rating,Genres
0,30,4.3,165299,5.400000,0,0.0,1,109
1,6,3.8,12667,34.000000,0,0.0,1,35
2,19,3.0,4,21.534596,0,0.0,1,69
3,21,4.3,355921,17.000000,0,0.0,4,74
4,14,4.2,3816,31.000000,0,0.0,4,6


In [30]:
X_train

,Category,Rating,Reviews,Size,Type,Price,Content Rating,Genres
0,0,4.1,159,19.000000,0,0.0,1,9
1,0,3.9,967,14.000000,0,0.0,1,12
3,0,4.5,215644,25.000000,0,0.0,4,9
4,0,4.3,967,2.800000,0,0.0,1,11
6,0,3.8,178,19.000000,0,0.0,1,9
...,...,...,...,...,...,...,...,...
10835,4,4.3,0,9.600000,0,0.0,1,22
10837,11,5.0,4,3.600000,0,0.0,1,38
10838,20,4.3,3,9.500000,0,0.0,1,70
10839,3,4.5,114,21.534596,0,0.0,3,19


In [31]:
y_train

0        0
1        1
3        1
4        0
6        0
        ..
10835    0
10837    0
10838    0
10839    0
10840    1
Name: Installs, Length: 7873, dtype: int32

In [32]:
#DECISION TREE

classify_DT = DecisionTreeClassifier(max_leaf_nodes = 29, random_state = 0) #max_leaf_nodes signifies no. of nodes
classify_DT.fit(X_train, y_train)
predict_popular_DT = classify_DT.predict(X_test)
print("Predicted: ",predict_popular_DT[:30])
print("Actual:    ",np.array(y_test[:30]))

Predicted:  [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 0]
Actual:     [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]


In [33]:
#RANDOM FOREST

classify_RF = RandomForestRegressor(n_estimators = 100 , random_state = 0)   #n_estimators is no of trees in forest
classify_RF.fit(X_train , y_train)
predict_popular_RF = classify_RF.predict(X_test)
print("Predicted: ",predict_popular_RF[:30].astype("int64"))
print("Actual:    ",np.array(y_test[:30]))

Predicted:  [1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 0 0 0 0 0 0 1 1 1 0]
Actual:     [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]


In [34]:
classify_LR1 = LinearRegression()
classify_LR1.fit(X_train , y_train)
predict_popular_LR1 = classify_LR1.predict(X_test)
print("Predicted: ",predict_popular_LR1[:30].astype("int64"))
print("Actual:    ",np.array(y_test[:30]))

Predicted:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Actual:     [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]


In [35]:
acc_LR1 = accuracy_score(y_true = y_test, y_pred = predict_popular_LR1.astype("int64"))
print("Accuracy of Decision Tree : ", (acc_LR1*100))

def Evaluationmatrix_LR1(y_test, y_pred):
    print ('Mean Squared Error: '+ str(metrics.mean_squared_error(y_test,y_pred)))
    print ('Mean absolute Error: '+ str(metrics.mean_absolute_error(y_test,y_pred)))

Evaluationmatrix_LR1(y_test,predict_popular_LR1)

print('R square value : ',r2_score(y_test, predict_popular_LR1))

Accuracy of Decision Tree :  43.41216216216216
Mean Squared Error: 0.23882712827528538
Mean absolute Error: 0.45068212591513557
R square value :  0.026312855576316796


In [36]:
#LOGISTIC REGRESSION

classify_LR = LogisticRegression(C=1e5, solver='lbfgs', multi_class='multinomial')
classify_LR.fit(X_train , y_train)
predict_popular_LR = classify_LR.predict(X_test)
print("Predicted: ",predict_popular_LR[:30].astype("int64"))
print("Actual:    ",np.array(y_test[:30]))

Predicted:  [1 1 0 1 0 0 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0]
Actual:     [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]


C:\Users\Vamsi Sai\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


In [37]:
#NAIVE BAYES

classify_NB = GaussianNB()
classify_NB.fit(X_train, y_train)
predict_popular_NB = classify_NB.predict(X_test)
print("Predicted: ",predict_popular_NB[:30].astype("int64"))
print("Actual:    ",np.array(y_test[:30]))

Predicted:  [1 1 0 1 0 0 0 1 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0]
Actual:     [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]


In [38]:
#K NEAREST NEIGHBORS

classify_KNN = KNeighborsClassifier(n_neighbors=15)
classify_KNN.fit(X_train, y_train)
predict_popular_KNN = classify_KNN.predict(X_test)
print("Predicted: ",predict_popular_KNN[:30].astype("int64"))
print("Actual:    ",np.array(y_test[:30]))

C:\Users\Vamsi Sai\AppData\Local\Programs\Python\Python37-32\lib\site-packages\sklearn\base.py:442: UserWarning: X does not have valid feature names, but KNeighborsClassifier was fitted with feature names
  "X does not have valid feature names, but"


Predicted:  [1 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0]
Actual:     [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]


In [39]:
#Artificial Neural Network

classify_ANN = MLPClassifier(solver='lbfgs', alpha=0.5, hidden_layer_sizes=(5, 3), random_state=0)
classify_ANN.fit(X_train, y_train)
predict_popular_ANN = classify_ANN.predict(X_test)
print("Predicted: ",predict_popular_ANN[:30].astype("int64"))
print("Actual:    ",np.array(y_test[:30]))

Predicted:  [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 0 0 0 1 1 1 0]
Actual:     [1 1 0 1 0 0 0 1 1 0 1 1 1 0 1 1 1 0 0 1 1 0 0 1 0 0 1 1 1 1]


In [40]:
acc_DT = accuracy_score(y_true = y_test, y_pred = predict_popular_DT)
print("Accuracy of Decision Tree :", (acc_DT*100).astype("int64"),"%")

Accuracy of Decision Tree : 96 %


In [41]:
acc_RF = accuracy_score(y_true = y_test, y_pred = predict_popular_RF.astype("int64"))
print("Accuracy of Random Forest : ", (acc_RF*100).astype("int64"),"%")

Accuracy of Random Forest :  85 %


In [42]:
acc_LR = accuracy_score(y_true = y_test, y_pred = predict_popular_LR.astype("int64"))
print("Accuracy of Linear Regression : ", (acc_LR*100).astype("int64").astype("int64"),"%")

Accuracy of Linear Regression :  94 %


In [43]:
acc_NB = accuracy_score(y_true = y_test, y_pred = predict_popular_NB.astype("int64"))
print("Accuracy of Naive Bayes : ", (acc_NB*100).astype("int64"),"%")

Accuracy of Naive Bayes :  89 %


In [44]:
acc_KNN = accuracy_score(y_true = y_test, y_pred = predict_popular_KNN.astype("int64"))
print("Accuracy of K NEAREST NEIGHBOR : ", (acc_KNN*100).astype("int64"),"%")

Accuracy of K NEAREST NEIGHBOR :  94 %


In [45]:
acc_ANN = accuracy_score(y_true = y_test, y_pred = predict_popular_ANN.astype("int64"))
print("Accuracy of K NEURAL NETWORKS : ", (acc_ANN*100).astype("int64"),"%")

Accuracy of K NEURAL NETWORKS :  95 %


In [46]:
X_testCopy = X_test.copy()
X_testCopy["Popular"] = predict_popular_DT
X_testCopy[X_test["Size"] == 21.516165].head()
X_testCopy.to_csv("Test.csv", encoding='utf-8', index=False)

In [47]:
X_testCopy

,Category,Rating,Reviews,Size,Type,Price,Content Rating,Genres,Popular
0,30,4.3,165299,5.400000,0,0.0,1,109,1
1,6,3.8,12667,34.000000,0,0.0,1,35,1
2,19,3.0,4,21.534596,0,0.0,1,69,0
3,21,4.3,355921,17.000000,0,0.0,4,74,1
4,14,4.2,3816,31.000000,0,0.0,4,6,0
...,...,...,...,...,...,...,...,...,...
1771,6,4.5,305218,21.534596,0,0.0,1,35,1
1772,12,4.4,337,3.300000,0,0.0,1,59,0
1773,11,4.0,1542,13.000000,0,0.0,1,51,0
1774,30,3.9,13275,71.000000,0,0.0,1,109,1


In [48]:
X_testCopy.sort_values(by='Popular').sample(10)

,Category,Rating,Reviews,Size,Type,Price,Content Rating,Genres,Popular
1506,12,1.0,2,7.200000,0,0.00,1,59,0
1292,11,4.2,632,50.000000,0,0.00,1,96,0
1507,11,4.3,616,25.000000,0,0.00,4,26,0
190,13,4.7,129737,21.534596,0,0.00,4,60,1
643,29,4.4,457,1.100000,1,2.99,1,107,0
1677,14,4.5,2032,50.000000,0,0.00,4,23,0
482,3,3.4,116,48.000000,1,4.60,1,19,0
1603,11,4.4,63,7.600000,0,0.00,1,51,0
128,11,4.3,382,15.000000,1,2.99,1,39,0
1349,20,4.3,2,1.800000,1,0.99,1,70,0
